# Final Project | How to predict a pandemic 
by Kevin Spurk

Text

# Table of content

1. 
2.
3.
...

# 1 | Setup

### 1.1. Libary imports

In [137]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('max_row', None)

### 1.2 Data imports

### 1.2.1 google mobility data

In [19]:
# csv import
data_mob_global = pd.read_csv('data/mobility_google/Global_Mobility_Report.csv')

# dict of countries and there languages to search in 
countries_lang = {'GB': 'en', 'DE': 'de', 'FR': 'fr', 'NL': 'nl',
                  'ES': 'es', 'PT': 'pt', 'PL': 'pl', 'IT': 'it',
                  'AT': 'de', 'DK': 'da', 'SE': 'sv', 'GR': 'el', 'CZ': 'cs'}

# getting subset of row with data of the target countries
tgt_countries = [country_code for country_code in countries_lang.keys()]
data_mob_eu = data_mob_global[data_mob_global['country_region_code'].isin(tgt_countries)]

len(data_mob_eu)


1363509

In [206]:
data_mob_eu.head(100)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
274011,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-15,9.0,1.0,42.0,13.0,0.0,-2.0
274012,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-16,15.0,21.0,42.0,12.0,1.0,-2.0
274013,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-17,9.0,5.0,35.0,3.0,-4.0,0.0
274014,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-18,8.0,5.0,40.0,2.0,-4.0,0.0
274015,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-19,4.0,2.0,10.0,-1.0,-5.0,1.0
274016,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-20,4.0,1.0,13.0,2.0,-5.0,0.0
274017,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-21,1.0,4.0,7.0,-2.0,-5.0,1.0
274018,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-22,9.0,3.0,52.0,14.0,2.0,-2.0
274019,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-23,4.0,23.0,4.0,5.0,2.0,0.0
274020,AT,Austria,NaN,NaN,NaN,NaN,NaN,ChIJfyqdJZsHbUcRr8Hk3XvUEhA,2020-02-24,9.0,10.0,25.0,3.0,2.0,-1.0


### 1.2.2 facebook mobility data

### 1.2.3 google search trends data

In [237]:
# connect to google and deepL APIs 
from pytrends.request import TrendReq
from pytrends import dailydata
from translate import Translator 

# get authentification for deepL API
with open('auth/deepl_auth_key.txt') as f:
    deepl_auth_key = f.read()

# google trends API access request
pytrends = TrendReq(hl='en-US', tz=0)

In [8]:
# keyword list 
keywords_list = ['covid', 'corona', 'SARS-CoV-2', 'virus', 'symptoms', 'fever', 'cough', 'tiredness', 'loss of smell', 'throat', 'shortness of breath', 'breathing issues', 'headache', 'chest pain', 'corona test', 'covid test', 'test', 'sick', 'infection']

# translating keyword list into target languages
from pyf.apis import deepl_translate_list

kw_dict, kw_df = deepl_translate_list(auth=deepl_auth_key, keywords=keywords_list, lang_out=countries_lang)


In [29]:
import time

# build payload
pytrends.build_payload(kw_list, cat=0, timeframe='2020-01-02 2021-11-02', geo='GB')


In [90]:
data4 = pytrends.get_historical_interest(kw_list, year_start=2020, month_start=1, day_start=2, hour_start=0, year_end=2021, month_end=11, day_end=2, hour_end=23, sleep=60)

In [154]:
data4

,date,cold
0,2020-01-02,65.041667
1,2020-01-03,66.041667
2,2020-01-04,69.250000
3,2020-01-05,68.416667
4,2020-01-06,65.083333
5,2020-01-07,64.791667
6,2020-01-08,58.708333
7,2020-01-09,28.600000
8,2020-01-10,27.791667
9,2020-01-11,27.958333


In [157]:
'''
while test_date_start <= test_date_end:
    if test_date_start not in data4['date']:
        print(test_date_start.strftime('%Y-%m-%d'))
        test_date_start += timedelta(days=1)
'''

'''
for i in data4['date']:
    if i == test_date:
        date_present = True
    else:
        pass
        
print(date_present)
'''

"\nfor i in data4['date']:\n    if i == test_date:\n        date_present = True\n    else:\n        pass\n        \nprint(date_present)\n"

In [ ]:
from datetime import date, timedelta

def g_trends_complete_timeframe(df, kw):
    # set start and end date for loop
    dt_s = df['date'][0]
    pos_end = len(df) - 1
    dt_e = df['date'][pos_end]
    
    # lists for missing days
    dates_missing = []
    trend_missing = []
    
    # loop to find missing dates in df
    while dt_s <= dt_e:
        date_check = False
        for d in df['date']:
            if d == dt_s:
                date_check = True
        # append missing date to lists
        if date_check == False:
            dates_missing.append(dt_s)
            trend_missing.append(np.NaN)
        dt_s += timedelta(days=1)
    
    # df for missing dates
    df_dates_missing = pd.DataFrame({'date': dates_missing, kw: trend_missing})
    df_completed = pd.concat([df, df_dates_missing], axis=0)
    df_completed['date'] = pd.to_datetime(df_completed['date'], errors='coerce')
    df_completed = df_completed.sort_values(by=['date'], ignore_index=True)
    
    return df_completed   


def g_trends_df_transform(df):
    # drop unwanted col
    df = df.drop(df.iloc[:, 1:], axis=1)
            
    # get date as col
    df.reset_index(inplace=True)
    df['date'] = df['date'].dt.date
    
    return df

def df_to_csv_format(df, name_cst, name_var, index=False):
    file_name = name_cst + '_' + name_var
    df.to_csv(f'{file_name}.csv', sep=',', index=index)
    return df


def g_trends_multilang(kw_dict, dt_start, dt_end, main_lang='GB', cat=0, sleep=60):
    # df for all results
    trends_all = pd.DataFrame({})
    
    # loop through lang
    for geo in kw_dict.keys():
        # df for results of one loc
        search_trend = pd.DataFrame({})
        
        # loop through keywords
        for k in kw_dict[geo]:
            # get df with search trend for kw
            k_trend = dailydata.get_daily_data(k, int(dt_start.split('-')[0]), int(dt_start.split('-')[1]), int(dt_end.split('-')[0]), int(dt_end.split('-')[1]), geo=geo)
            
            # transform df
            k_trend = g_trends_df_transform(k_trend)
            
            # add missing dates
            k_trend_completed = g_trends_complete_timeframe(df=k_trend, kw=k)
            
            # add df with search trend for kw to df for results of one loc
            search_trend['date'] = k_trend_completed['date']
            search_trend[k] = k_trend_completed[k]
        
        # add loc to df 
        search_trend['location'] = geo
        # export df to csv
        search_details = dt_start + '_' + dt_end + geo
        df_to_csv_format(df=search_trend, name_cst='g_trend', name_var=search_details, index=False)
        
        # concat df with df holding all results
        if len(trends_all) > 0:
            search_details.columns = trends_all.columns
        trends_all = pd.concat([trends_all, search_details], axis=0)
    
    return trends_all


In [228]:
from datetime import date, timedelta

def g_trends_complete_timeframe(df, dt_start, dt_end, kw_listed):
    
    # convert input to datetime
    dt_s = datetime.datetime.strptime(dt_start, '%Y-%m-%d').date()
    dt_e = datetime.datetime.strptime(dt_end, '%Y-%m-%d').date()
    # lists for missing days
    dates_missing = []
    trend_missing = []
    
    # loop to find missing dates in df
    while dt_s <= dt_e:
        date_check = False
        for d in df['date']:
            if d == dt_s:
                date_check = True
        # append missing date to lists
        if date_check == False:
            dates_missing.append(dt_s)
            trend_missing.append(np.NaN)
        dt_s += timedelta(days=1)
    
    # df for missing dates
    df_dates_missing = pd.DataFrame({'date': dates_missing, kw_listed[0]: trend_missing})
    df_completed = pd.concat([df, df_dates_missing], axis=0)
    df_completed['date'] = pd.to_datetime(df_completed['date'], errors='coerce')
    df_completed = df_completed.sort_values(by=['date'], ignore_index=True)
    
    return df_completed   


def g_trends_df_transform(df):
    # drop unwanted col
    for col in df.columns:
        if col == 'isPartial':
            df = df.drop(['isPartial'], axis=1)
            
    # get date as col
    df.reset_index(inplace=True)
    df['date'] = df['date'].dt.date
    
    # group rows by date
    df = df.groupby(['date']).mean()
    df.reset_index(inplace=True)
    df['date'] = df['date'].dt.date
    
    return df

def df_to_csv_format(df, name_cst, name_var, index=False):
    file_name = name_cst + '_' + name_var
    df.to_csv(f'{file_name}.csv', sep=',', index=index)
    return df


def g_trends_multilang(kw_dict, dt_start, dt_end, main_lang='GB', cat=0, sleep=60):
    # df for all results
    trends_all = pd.DataFrame({})
    
    # build timeframe str for api
    timeframe = dt_start + ' ' + dt_end
    
    # loop through lang
    for geo in kw_dict.keys():
        # df for results of one loc
        search_trend = pd.DataFrame({})
        # reformat kw list for sub func
        kw_group = list(zip(*[iter(kw_dict[geo])]*1))
        kw_listlist = [list(kw) for kw in kw_group]
        
        # loop through keywords
        for k in kw_listlist:
            # build payload
            print(k)
            pytrends.build_payload(kw_list=k, cat=cat, timeframe=timeframe, geo=geo)
            # get df with search trend for kw
            k_trend = pytrends.get_historical_interest(k, year_start=int(dt_start.split('-')[0]), month_start=int(dt_start.split('-')[1]), day_start=int(dt_start.split('-')[2]), hour_start=0, year_end=int(dt_end.split('-')[0]), month_end=int(dt_end.split('-')[1]), day_end=int(dt_end.split('-')[2]), hour_end=23, sleep=sleep)
            
            print(k_trend.head())
            
            # transform df
            k_trend = g_trends_df_transform(k_trend)
            
            # add missing dates
            k_trend_completed = g_trends_complete_timeframe(df=k_trend, dt_start=dt_start, dt_end=dt_end, kw_listed=k)
            
            # add df with search trend for kw to df for results of one loc
            search_trend['date'] = k_trend_completed['date']
            search_trend[k[0]] = k_trend_completed[k[0]]
        
        # add loc to df 
        search_trend['loc'] = geo
        # export df to csv
        search_details = dt_start + '_' + dt_end + geo
        df_to_csv_format(df=search_trend, name_cst='g_trend', name_var=search_details, index=False)
        # concat df with df holding all results
        if len(trends_all) > 0:
            search_details.columns = trends_all.columns
        trends_all = pd.concat([trends_all, search_details], axis=0)
    
    return trends_all


In [244]:
data44 = data4.copy()

for col in data44.columns:
    if col == 'cold':
        data44 = data44.drop(['cold'], axis=1)
        
data44['date'][-1]

KeyError: -1

In [232]:
test0_lang = {'GB': 'en', 'DK': 'da', 'GR': 'el'}
test0_kw = ['germany']#, 'Tesla', 'fever']

In [238]:
dailydata.get_daily_data(test0_kw[0], 2020, 1, 2021, 10, geo = 'GB')

germany:2020-01-01 2020-01-31
germany:2020-02-01 2020-02-29
germany:2020-03-01 2020-03-31
germany:2020-04-01 2020-04-30
germany:2020-05-01 2020-05-31
germany:2020-06-01 2020-06-30
germany:2020-07-01 2020-07-31
germany:2020-08-01 2020-08-31
germany:2020-09-01 2020-09-30
germany:2020-10-01 2020-10-31
germany:2020-11-01 2020-11-30
germany:2020-12-01 2020-12-31
germany:2021-01-01 2021-01-31
germany:2021-02-01 2021-02-28
germany:2021-03-01 2021-03-31
germany:2021-04-01 2021-04-30
germany:2021-05-01 2021-05-31
germany:2021-06-01 2021-06-30
germany:2021-07-01 2021-07-31
germany:2021-08-01 2021-08-31
germany:2021-09-01 2021-09-30
germany:2021-10-01 2021-10-31


,germany_unscaled,germany_monthly,isPartial,scale,germany
date,,,,,
2020-01-01,69,NaN,NaN,NaN,NaN
2020-01-02,69,NaN,NaN,NaN,NaN
2020-01-03,56,NaN,NaN,NaN,NaN
2020-01-04,81,NaN,NaN,NaN,NaN
2020-01-05,62,7.0,False,0.07,4.34
2020-01-06,77,7.0,NaN,0.07,5.39
2020-01-07,72,7.0,NaN,0.07,5.04
2020-01-08,68,7.0,NaN,0.07,4.76
2020-01-09,63,7.0,NaN,0.07,4.41


In [235]:
# build payload
pytrends.build_payload(kw_list=test0_kw, cat=0, timeframe='2020-01-02 2021-11-02', geo='GB')

In [214]:
test0_dict, test0_df = deepl_translate_list(auth=deepl_auth_key, keywords=test0_kw, lang_out=test0_lang)

In [215]:
test0_df

,GB,DK,GR
0,germany,tyskland,Γερμανία
1,Tesla,Tesla,Tesla
2,fever,feber,πυρετός


In [242]:
df_new = test0_df.drop(test0_df.iloc[:, 1:], axis = 1)
df_new

,GB
0,germany
1,Tesla
2,fever


In [229]:
test0_trends = g_trends_multilang(kw_dict=test0_dict, dt_start='2021-08-01', dt_end='2021-08-21', main_lang='GB')

['germany']
Empty DataFrame
Columns: []
Index: []


KeyError: 'date'

In [203]:
date_t1 = datetime.datetime.strptime('2020-07-06', '%Y-%m-%d').date()
exists = date_t1 in data4['date']
exists

False

In [30]:
data5 = pytrends.related_queries()

In [31]:
data5

{'cold': {'top':                      query  value
  0                 cold war    100
  1                cold sore     31
  2            cold symptoms     24
  3                cold call     21
  4                cold feet     20
  5             call of duty     18
  6    cold war call of duty     17
  7             hot and cold     14
  8               cold sores     14
  9             the cold war     13
  10      black ops cold war     11
  11            cod cold war     11
  12               baby cold     11
  13    cold weather payment     11
  14            feeling cold     10
  15             common cold      9
  16        cold war zombies      9
  17              stone cold      9
  18              cold hands      8
  19            cold and flu      8
  20     cold symptoms covid      7
  21               head cold      7
  22          covid symptoms      7
  23  how to get rid of cold      6
  24             cold shower      6,
  'rising':                                  que

In [ ]:
data3 = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
data3

In [ ]:
data6 = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
data6

In [ ]:
#1 Interest over Time
'''
data = pytrends.interest_over_time() 
data = data.reset_index() 

#1 Interest over Time plot

import plotly.express as px

fig = px.line(data, x="date", y=['covid'], title='Keyword Web Search Interest Over Time')
fig.show() '''

In [ ]:
len(data)#.head()

In [ ]:
data